<a href="https://colab.research.google.com/github/Kaleab1999/Medical-Transcription-for-medicne-department/blob/main/Biobert_for_MT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.6 MB/s eta 0:00:00


In [2]:
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
df= pd.read_csv("/content/preproceed MT file.csv")
df.head()

,Unnamed: 0,transcription,medical_specialty,tokenised,stemmed,stemmed_without_stop
0,3,d m mode left atrial enlargement with left a...,Cardiovascular / Pulmonary,"['d', 'm', 'mode', 'left', 'atrial', 'enlargem...","['d', 'm', 'mode', 'left', 'atrial', 'enlarg',...",mode left atrial enlarg left atrial diamet cm ...
1,4,the left ventricular cavity size and wall thi...,Cardiovascular / Pulmonary,"['the', 'left', 'ventricular', 'cavity', 'size...","['the', 'left', 'ventricular', 'caviti', 'size...",left ventricular caviti size wall thick appear...
2,7,d echocardiogram multiple views of the heart ...,Cardiovascular / Pulmonary,"['d', 'echocardiogram', 'multiple', 'views', '...","['d', 'echocardiogram', 'multipl', 'view', 'of...",echocardiogram multipl view heart great vessel...
3,9,description normal cardiac chambers size nor...,Cardiovascular / Pulmonary,"['description', 'normal', 'cardiac', 'chambers...","['descript', 'normal', 'cardiac', 'chamber', '...",descript normal cardiac chamber size normal le...
4,11,d study mild aortic stenosis widely calcifie...,Cardiovascular / Pulmonary,"['d', 'study', 'mild', 'aortic', 'stenosis', '...","['d', 'studi', 'mild', 'aortic', 'stenosi', 'w...",studi mild aortic stenosi wide calcifi minim r...


In [4]:
model_name = 'dmis-lab/biobert-base-cased-v1.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [5]:
df['medical_specialty'].value_counts()

 Cardiovascular / Pulmonary    371
 General Medicine              259
 Gastroenterology              224
 Neurology                     223
Name: medical_specialty, dtype: int64

In [6]:
df.shape

(1077, 6)

In [7]:
data= df[['stemmed_without_stop','medical_specialty']]
data.head()

,stemmed_without_stop,medical_specialty
0,mode left atrial enlarg left atrial diamet cm ...,Cardiovascular / Pulmonary
1,left ventricular caviti size wall thick appear...,Cardiovascular / Pulmonary
2,echocardiogram multipl view heart great vessel...,Cardiovascular / Pulmonary
3,descript normal cardiac chamber size normal le...,Cardiovascular / Pulmonary
4,studi mild aortic stenosi wide calcifi minim r...,Cardiovascular / Pulmonary


In [8]:
data.isnull().sum()

stemmed_without_stop    0
medical_specialty       0
dtype: int64

In [9]:
data.iloc[[6]]

,stemmed_without_stop,medical_specialty
6,histori present ill patient year old femal pre...,General Medicine


In [10]:
X= data['stemmed_without_stop'].tolist()
y= list( data['medical_specialty'].astype('category').cat.codes)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2)

### model building

In [12]:
train_d= tokenizer(X_train, padding= True, truncation= True, max_length=512)
test_d= tokenizer(X_test, padding= True, truncation= True, max_length=512)

In [13]:
train_d.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

#### create dataset

In [14]:
import torch

In [15]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [16]:
train_dataset = Dataset(train_d, y_train)
test_dataset = Dataset(test_d, y_test)

In [17]:
train_dataset[5]

{'input_ids': tensor([  101,  2548,  5351,  1352,  1631,  1618,   178,  1964,  1821,  2660,
          7858,  1320, 18463,  8517,  4844, 12647, 12736,  2047, 21828,  4907,
          5552, 27629,  8992, 10542,  1465,  1844, 28057,  1647,  1525,  1113,
          2646,  9712, 14093,  4195, 27629,  8992, 10542,  1465,  2043,  2703,
          1562,  6322,  3189, 28057,   174,  1377,  1403,  8332,  1437,  4142,
          2703,  1120, 13119, 20497, 27647,  1233,  4844, 10454, 11850,  1361,
          6795,  5351,  1352,  1989,  1313,  2112,  4184,   170, 12148,  1596,
           191,  1348,  1964,  1231,  1643,  1742,  1665,   170,  1830,  1665,
          1143, 13328,  2057,   170, 12148,  1596,   188,  5208,  2155,  1182,
          1248,  7710, 14255,  4915,  2875, 16516,  6697, 25786,   191,  1348,
          1964,  3532, 13590,  1352,  1603,  2184,  4252,  7340, 19428,  1233,
          1870, 24034, 12316,  3073, 19807, 14255, 17903,  1126,  1182,  1260,
          1665,  2836,  2112,  4184,  1

In [18]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    # average= 'micro' because it's multi-class classification u can use also macro, or weight or other 
    # u shouldn't mention anything for binary classification
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')

    return { "precision": precision, "recall": recall, 'f1-score': f1, "acc": accuracy}

In [19]:
# Define Trainer
args = TrainingArguments(
    output_dir="result1",
    num_train_epochs=5,
    per_device_train_batch_size=16

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [20]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 861
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 270
  Number of trainable parameters = 108313348


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=270, training_loss=0.42795158668800637, metrics={'train_runtime': 415.2945, 'train_samples_per_second': 10.366, 'train_steps_per_second': 0.65, 'total_flos': 1132713433313280.0, 'train_loss': 0.42795158668800637, 'epoch': 5.0})

In [21]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 216
  Batch size = 8


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.4764449894428253,
 'eval_precision': 0.875,
 'eval_recall': 0.875,
 'eval_f1-score': 0.875,
 'eval_acc': 0.875,
 'eval_runtime': 7.3654,
 'eval_samples_per_second': 29.326,
 'eval_steps_per_second': 3.666,
 'epoch': 5.0}